In [ ]:
#Loading in Packages and Data

#Importing Packages
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
import xarray as xr
import os; import time
import pickle
import h5py
###############################################################
def coefs(coefficients,degree):
    coef=coefficients
    coefs=""
    for n in range(degree, -1, -1):
        string=f"({coefficients[len(coef)-(n+1)]:.1e})"
        coefs+=string + f"x^{n}"
        if n != 0:
            coefs+=" + "
    return coefs
###############################################################

# Importing Model Data
check=False
dir='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'

# # dx = 1 km; Np = 1M; Nt = 5 min
# data=xr.open_dataset(dir+'../cm1r20.3/run/cm1out_1km_5min.nc') #***
# parcel=xr.open_dataset(dir+'../cm1r20.3/run/cm1out_pdata_1km_5min_1e6.nc') #***
# res='1km';t_res='5min'
# Np_str='1e6'

# dx = 1km; Np = 50M
#Importing Model Data
check=False
dir2='/home/air673/koa_scratch/'
data=xr.open_dataset(dir2+'cm1out_1km_1min.nc') #***
parcel=xr.open_dataset(dir2+'cm1out_pdata_1km_1min_50M.nc') #***
res='1km'; t_res='1min'; Np_str='50e6'

# # dx = 1km; Np = 100M
# #Importing Model Data
# check=False
# dir2='/home/air673/koa_scratch/'
# data=xr.open_dataset(dir2+'cm1out_1km_1min.nc') #***
# parcel=xr.open_dataset(dir2+'cm1out_pdata_1km_1min_100M.nc') #***
# res='1km'; t_res='1min'; Np_str='100e6'

# #uncomment if using 250m data
# #Importing Model Data
# check=False
# dir2='/home/air673/koa_scratch/'
# data=xr.open_dataset(dir2+'cm1out_250m.nc') #***
# # # parcel=xr.open_dataset(dir2+'cm1out_pdata_250m.nc') #***

# # Restricts the timesteps of the data from timesteps0 to 140
# data=data.isel(time=np.arange(0,400+1))
# # # parcel=parcel.isel(time=np.arange(0,400+1))
# res='250m'

times=data['time'].values/(1e9 * 60); times=times.astype(float);
minutes=1/times[1] #1 / minutes per timestep = timesteps per minute

In [ ]:
import sys
dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
path=dir2+'../Functions/'
sys.path.append(path)

import NumericalFunctions
from NumericalFunctions import * # import NumericalFunctions 
import PlottingFunctions
from PlottingFunctions import * # import PlottingFunctions


# # Get all functions in NumericalFunctions
# import inspect
# functions = [f[0] for f in inspect.getmembers(NumericalFunctions, inspect.isfunction)]
# functions

# # Get all functions in NumericalFunctions
# import inspect
# functions = [f[0] for f in inspect.getmembers(PlottingFunctions, inspect.isfunction)]
# functions

In [ ]:
def check_memory():
    import sys
    ipython_vars = ["In", "Out", "exit", "quit", "get_ipython", "ipython_vars"]
    print("Top 10 objects with highest memory usage")
    # Get a sorted list of the objects and their sizes
    mem = {
        key: round(value/1e6,2)
        for key, value in sorted(
            [
                (x, sys.getsizeof(globals().get(x)))
                for x in globals()
                if not x.startswith("_") and x not in sys.modules and x not in ipython_vars
            ],
            key=lambda x: x[1],
            reverse=True)[:10]
    }
    print({key:f"{value} MB" for key,value in mem.items()})
    print(f"\n{round(sum(mem.values()),2)/1000} GB in use overall")

In [ ]:
#JOB ARRAY SETUP
job_array=False;index_adjust=0
job_array=True
##############################*#*

if job_array==True:
    num_jobs=300 #how many total jobs are being run? i.e. array=1-100 ==> num_jobs=100 #***
    total_elements=len(parcel['xh']) #total num of variables

    if num_jobs >= total_elements:
        raise ValueError("Number of jobs cannot be greater than or equal to total elements.")
    
    job_range = total_elements // num_jobs  # Base size for each chunk
    remaining = total_elements % num_jobs   # Number of chunks with 1 extra 
    
    # Function to compute the start and end for each job_id
    def get_job_range(job_id, num_jobs):
        job_id-=1
        # Add one extra element to the first 'remaining' chunks
        start_job = job_id * job_range + min(job_id, remaining)
        end_job = start_job + job_range + (1 if job_id < remaining else 0)
    
        if job_id == num_jobs - 1: 
            end_job = total_elements #- 1
        return start_job, end_job
    # def job_testing():
    #     #TESTING
    #     start=[];end=[]
    #     for job_id in range(1,num_jobs+1):
    #         start_job, end_job = get_job_range(job_id)
    #         print(start_job,end_job)
    #         start.append(start_job)
    #         end.append(end_job)
    #     print(np.all(start!=end))
    #     print(len(np.unique(start))==len(start))
    #     print(len(np.unique(end))==len(end))
    # job_testing()
    
    job_id = int(os.environ.get('SLURM_ARRAY_TASK_ID', 0)) #this is the current SBATCH job id
    if job_id==0: job_id=1
    start_job, end_job = get_job_range(job_id, num_jobs)
    index_adjust=start_job
    print(f'start_job = {start_job}, end_job = {end_job}')

In [ ]:
if job_array==True:
    #Indexing Array with JobArray
    parcel=parcel.isel(xh=slice(start_job,end_job))
    #(for 150_000_000 parcels use 500-1000 jobs)

In [ ]:
# Reading Back Data Later
##############
def make_data_dict(in_file,var_names,read_type):
    if read_type=='h5py':
        with h5py.File(in_file, 'r') as f:
            if job_array==True:
                data_dict = {var_name: f[var_name][:,start_job:end_job] for var_name in var_names}
            elif job_array==False:
                data_dict = {var_name: f[var_name][:] for var_name in var_names}
            
    elif read_type=='xarray':
        in_data = xr.open_dataset(
            in_file,
            engine='h5netcdf',
            phony_dims='sort',
            chunks={'phony_dim_0': 100, 'phony_dim_1': 1_000_000} 
        )
        if job_array==True:
            data_dict = {k: in_data[k][:,start_job:end_job].compute().data for k in var_names}
        elif job_array==False:
            data_dict = {k: in_data[k][:].compute().data for k in var_names}
    return data_dict

# read_type='xarray'
read_type='h5py'

In [ ]:
import h5py
dir2=dir+'Project_Algorithms/Lagrangian_Arrays/'
in_file=dir2+f'lagrangian_binary_array_{res}_{t_res}_{Np_str}.h5'

var_names = ['W', 'QCQI', 'Z', 'Y', 'X', 'z']
data_dict = make_data_dict(in_file,var_names,read_type)
W, QCQI, Z, Y, X, parcel_z = (data_dict[k] for k in var_names)

# #Making Time Matrix
# rows, cols = A.shape[0], A.shape[1]
# T = np.arange(rows).reshape(-1, 1) * np.ones((1, cols), dtype=int)
check_memory()

In [ ]:
################################################################################

In [ ]:
########################
#READING BACK IN
def LoadFinalData(in_file):
    dict = {}
    with h5py.File(in_file, 'r') as f:
        for key in f.keys():
            dict[key] = f[key][:]
    return dict

def LoadAllCloudBase():
    dir2 = dir + f'Project_Algorithms/Tracking_Algorithms/'
    in_file = dir2 + f"all_cloudbase_{res}_{t_res}_{Np_str}.pkl"
    with open(in_file, 'rb') as f:
        all_cloudbase = pickle.load(f)
    return(all_cloudbase)
min_all_cloudbase=np.nanmin(LoadAllCloudBase())
print(f"Minimum Cloudbase is: {min_all_cloudbase}\n")

dir2 = dir + f'Project_Algorithms/Tracking_Algorithms/'
in_file=dir2+f"parcel_tracking_SUBSET_{res}_{t_res}_{Np_str}"
final_dict=LoadFinalData(in_file)


#DYNAMICALLY CREATING VARIABLES
for key, value in final_dict.items():
    globals()[key] = value

# #DYNAMICALLY PRINTING VARIABLE SIZES
# for key in final_dict:
#     print(f"{key} has {final_dict[key].shape[0]} parcels")

# PRINTING VARIABLE SIZES (ONE BY ONE)
print(f'ALL: {len(CL_ALL_out_arr)} CL parcels and {len(nonCL_ALL_out_arr)} nonCL parcels')
print(f'SHALLOW: {len(CL_SHALLOW_out_arr)} CL parcels and {len(nonCL_SHALLOW_out_arr)} nonCL parcels')
print(f'DEEP: {len(CL_DEEP_out_arr)} CL parcels and {len(nonCL_DEEP_out_arr)} nonCL parcels')
print('\n')
print(f'ALL: {len(SBZ_ALL_out_arr)} SBZ parcels and {len(nonSBZ_ALL_out_arr)} nonSBZ parcels')
print(f'SHALLOW: {len(SBZ_SHALLOW_out_arr)} SBZ parcels and {len(nonSBZ_SHALLOW_out_arr)} nonSBZ parcels')
print(f'DEEP: {len(SBZ_DEEP_out_arr)} SBZ parcels and {len(nonSBZ_DEEP_out_arr)} nonSBZ parcels')
print('\n')
print(f'ALL: {len(ColdPool_ALL_out_arr)} ColdPool parcels')
print(f'SHALLOW: {len(ColdPool_SHALLOW_out_arr)} ColdPool parcels')
print(f'DEEP: {len(ColdPool_DEEP_out_arr)} ColdPool parcels')


#APPLYING JOB ARRAY
if job_array==True:
    print('APPLYING JOB ARRAY')
    def job_filter(arr):
        return arr[(arr[:,0]>=start_job)&(arr[:,0]<end_job)]
    for name in [
        'CL_ALL_out_arr', 'nonCL_ALL_out_arr',
        'CL_SHALLOW_out_arr', 'nonCL_SHALLOW_out_arr',
        'CL_DEEP_out_arr', 'nonCL_DEEP_out_arr',
        'SBZ_ALL_out_arr', 'nonSBZ_ALL_out_arr',
        'SBZ_SHALLOW_out_arr', 'nonSBZ_SHALLOW_out_arr',
        'SBZ_DEEP_out_arr', 'nonSBZ_DEEP_out_arr',
        'ColdPool_ALL_out_arr', 'ColdPool_SHALLOW_out_arr', 'ColdPool_DEEP_out_arr'
    ]:
        globals()[name] = job_filter(globals()[name])

In [ ]:
#ALL/DEEP/SHALLOW CL vs non-CL Tracked Parcel Plots
################################################################################

In [ ]:
# # TOO SLOW TO LOAD INTO MEMORY (TESTING WITH F(t))

# # Loading Important Variables
# ##############
# if 'emptylike' not in globals():
#     print('loading neccessary variables')
#     variable='winterp'; w_data=data[variable].data #get w data
#     print('done')
#     empty_like=True
# check_memory()

# import dask.array as da
# from dask.diagnostics import ProgressBar

# # Ensure xarray automatically handles the Dask array
# variable = 'qc'; qc_data = data[variable]  # Get qc data
# variable = 'qi'; qi_data = data[variable]  # Get qi data

# chunk_sizes = {'time': 5}  # Chunk only on time, leave others unchunked
# qc_data = qc_data.chunk(chunk_sizes)
# qi_data = qi_data.chunk(chunk_sizes)

# # Perform the computation
# qc_plus_qi = qc_data + qi_data  # Dask handles this efficiently

# # Enable progress bar and compute
# with ProgressBar():
#     qc_plus_qi = qc_plus_qi.compute()

def call_variables(t,z,y,x):
    t = t if t is not None else slice(None)
    z = z if z is not None else slice(None)
    y = y if y is not None else slice(None)
    x = x if x is not None else slice(None)
    
    variable='winterp'; w_data=data[variable].isel(time=t,zh=z,yh=y,xh=x).data
    variable='qc'; qc_data=data[variable].isel(time=t,zh=z,yh=y,xh=x).data
    variable='qi'; qi_data=data[variable].isel(time=t,zh=z,yh=y,xh=x).data
    qc_plus_qi=qc_data+qi_data
    return w_data,qc_plus_qi

# t=100
# z=10
# y=100
# x=400
# [x_slice,x_slice2]=call_variables(t,z,y,None)
# x_slice

In [ ]:
#MAKING PROFILE ARRAY

#thresholds
w_thresh1=0.1
w_thresh2=0.5
qcqi_thresh=1e-6

dx=(data['xf'][1].item()-data['xf'][0].item())# km *1e3 #meters
dy=(data['yf'][1].item()-data['yf'][0].item())# km *1e3 #meters

def tracked_AREA(type,type2,updraft_type):
    out_arr = globals()[f"{type2}_{type.upper()}_out_arr"].copy()
   
    zhs=data['zh'].values
    profile_array =np.zeros((len(zhs), 3)) #column 1: var, column 2: counter, column 3: list of zhs
    profile_array[:,2]=zhs;
    
    for row in range(out_arr.shape[0]):
        after=out_arr[row,3]
        if np.mod(row,50)==0: print(f'{row}/{out_arr.shape[0]}')
        p=out_arr[row,0]
        
        # ts=np.arange(out_arr[row,4],out_arr[row,5]+1 + after)
        ts_end = min(out_arr[row, 2] + 1 + after, len(data['time'])) #this takes care of exceeding buffers
        ts = np.arange(out_arr[row, 1], ts_end)
        
        zs=Z[ts,p-index_adjust] #JOBARRAY INDEXADJUST
        ys=Y[ts,p-index_adjust] #JOBARRAY INDEXADJUST
        xs=X[ts,p-index_adjust] #JOBARRAY INDEXADJUST

        for t, z, y, x in zip(ts, zs, ys, xs):

            if True==True: #x in np.arange(15,500) and y in np.arange(15,85): (OLD) USED TO AVOID BOUNDARIES
    
                #CALCULATING AREA (CURRENTLY IGNORES CASES WHERE UPDRAFT CROSSES THE BOUNDARY)
                ########################################################
                
                #FINDING XLENGTH
                # x_slice=w_data[t,z,y,:]
                # x_slice2=qc_plus_qi[t,z,y,:]
                [x_slice,x_slice2]=call_variables(t,z,y,None)#JOBARRAY
                
                
                if updraft_type=='general':
                    # x_slice[(x_slice >= w_thresh1)] = 1; x_slice[x_slice < w_thresh1] = 0
                    x_slice = np.where(x_slice >= w_thresh1, 1, 0)
                elif updraft_type=='cloudy':
                    # x_slice[(x_slice>=w_thresh2)&(x_slice2>=qcqi_thresh)]=1; x_slice[(x_slice<w_thresh2)|(x_slice2<qcqi_thresh)]=0
                    x_slice = np.where((x_slice >= w_thresh2) & (x_slice2 >= qcqi_thresh), 1, 0)

                if np.all(x_slice[x+1:]==1) or np.all(x_slice[:(x-1)+1]==1): #*AVOID BOUNDARY CASES*
                    continue #*AVOID BOUNDARY CASES*
                # x_right=np.where(x_slice[x+1:]==0)[0][0]+(x+1)
                # x_left=np.where(x_slice[:(x-1)+1]==0)[0][-1]
                x_right=np.where(x_slice[x:]==0)[0][0]+(x)
                x_left=np.where(x_slice[:(x)+1]==0)[0][-1]
                x_length=(x_right-x_left)*dx
                
                #FINDING YLENGTH
                # y_slice=w_data[t,z,:,x]
                # y_slice2=qc_plus_qi[t,z,:,x]
                [y_slice,y_slice2]=call_variables(t,z,None,x) #JOBARRAY
                
                if updraft_type=='general':
                    # y_slice[(y_slice>=w_thresh1)]=1; y_slice[y_slice<w_thresh1]=0
                    y_slice = np.where(y_slice >= w_thresh1, 1, 0)
                elif updraft_type=='cloudy':
                    y_slice = np.where((y_slice >= w_thresh2) & (y_slice2 >= qcqi_thresh), 1, 0)

                if np.all(y_slice[y+1:]==1) or np.all(y_slice[:(y-1)+1]==1): #*TOO AVOID BOUNDARY CASES*
                    continue #*TOO AVOID BOUNDARY CASES*
                # y_right=np.where(y_slice[y+1:]==0)[0][0]+(y+1)
                # y_left=np.where(y_slice[:(y-1)+1]==0)[0][-1]
                y_right=np.where(y_slice[y:]==0)[0][0]+(y)
                y_left=np.where(y_slice[:(y)+1]==0)[0][-1]
                y_length=(y_right-y_left)*dy
                
                AREA=x_length*y_length#square area approximation. 
                AREA*=np.pi/4 #include for oval area approximation
                ########################################################
            else:
                continue

            #ADDING TO PROFILE
            var=AREA
            profile_array[z,0]+=var;profile_array[z,1]+=1
    return profile_array

In [ ]:
######################
#CALCULATING

In [ ]:
print('ALL')
CL_ALL_profile_array_AREA_general=tracked_AREA(type='all',type2='CL',updraft_type='general')
CL_ALL_profile_array_AREA_cloudy=tracked_AREA(type='all',type2='CL',updraft_type='cloudy')
nonCL_ALL_profile_array_AREA_general=tracked_AREA(type='all',type2='nonCL',updraft_type='general')
nonCL_ALL_profile_array_AREA_cloudy=tracked_AREA(type='all',type2='nonCL',updraft_type='cloudy')


In [ ]:
print('SHALLOW')
CL_SHALLOW_profile_array_AREA_general=tracked_AREA(type='shallow',type2='CL',updraft_type='general')
CL_SHALLOW_profile_array_AREA_cloudy=tracked_AREA(type='shallow',type2='CL',updraft_type='cloudy')

nonCL_SHALLOW_profile_array_AREA_general=tracked_AREA(type='shallow',type2='nonCL',updraft_type='general')
nonCL_SHALLOW_profile_array_AREA_cloudy=tracked_AREA(type='shallow',type2='nonCL',updraft_type='cloudy')

In [ ]:
print('DEEP')
CL_DEEP_profile_array_AREA_general=tracked_AREA(type='deep',type2='CL',updraft_type='general')
CL_DEEP_profile_array_AREA_cloudy=tracked_AREA(type='deep',type2='CL',updraft_type='cloudy')

nonCL_DEEP_profile_array_AREA_general=tracked_AREA(type='deep',type2='nonCL',updraft_type='general')
nonCL_DEEP_profile_array_AREA_cloudy=tracked_AREA(type='deep',type2='nonCL',updraft_type='cloudy')

In [ ]:
#SAVING

profile_dict = {}

categories = ['ALL', 'SHALLOW', 'DEEP']
types2 = ['CL', 'nonCL']
updraft_types = ['general', 'cloudy']

for category in categories:
    print(category)  # Print category name
    for type2 in types2:
        for updraft_type in updraft_types:
            var_name = f"{type2}_{category}_profile_array_AREA_{updraft_type}"
            profile_dict[var_name] = eval(var_name)  # Dynamically reference existing variables

dir2=dir+'Project_Algorithms/Tracked_Profiles/job_out2/'
output_file=dir2+f"CL_nonCL_tracked_Area_profiles_{res}_{t_res}_{Np_str}"
if job_array==True:
    output_file+=f"_{job_id}.h5"
elif job_array==False:
    output_file+=f".h5"
with open(output_file, "wb") as f:
    pickle.dump(profile_dict, f)

In [ ]:
#########################################
#RECOMBINE SEPERATE JOB_ARRAYS AFTER
recombine=False #KEEP FALSE WHEN JOBARRAY IS RUNNING
# recombine=True

In [ ]:
if recombine==True:
    def load_dict(job_id):
        dir2=dir+'Project_Algorithms/Tracked_Profiles/job_out2/'
        input_file=dir2+f"CL_nonCL_tracked_Area_profiles_{res}_{t_res}_{Np_str}_{job_id}.h5"
        with open(input_file, "rb") as f:
            profile_dict = pickle.load(f)
        return profile_dict
    
    job_id=1
    profile_dict1 = load_dict(job_id)
    
    num_jobs=300
    for job_id in np.arange(1,num_jobs+1):
        profile_dict2 = load_dict(job_id)
        if np.mod(job_id,10)==0: print(f"job_id = {job_id}")
        for var in profile_dict2:
            profile_dict1[var][:,0:1+1]+=profile_dict2[var][:,0:1+1]
    
    dir2=dir+'Project_Algorithms/Tracked_Profiles/OUTPUT_FILES/'
    output_file=dir2+f"CL_nonCL_tracked_Area_profiles_{res}_{t_res}_{Np_str}.h5"
    
    with open(output_file, "wb") as f:
        pickle.dump(profile_dict1, f)

In [ ]:
############################################################
#SBZ

In [ ]:
################################
#CALCULATING

In [ ]:
print('\n'+'ALL')
SBZ_ALL_profile_array_AREA_general=tracked_AREA(type='all',type2='SBZ',updraft_type='general')
SBZ_ALL_profile_array_AREA_cloudy=tracked_AREA(type='all',type2='SBZ',updraft_type='cloudy')
nonSBZ_ALL_profile_array_AREA_general=tracked_AREA(type='all',type2='nonSBZ',updraft_type='general')
nonSBZ_ALL_profile_array_AREA_cloudy=tracked_AREA(type='all',type2='nonSBZ',updraft_type='cloudy')

print('\n'+'SHALLOW')
SBZ_SHALLOW_profile_array_AREA_general=tracked_AREA(type='shallow',type2='SBZ',updraft_type='general')
SBZ_SHALLOW_profile_array_AREA_cloudy=tracked_AREA(type='shallow',type2='SBZ',updraft_type='cloudy')

nonSBZ_SHALLOW_profile_array_AREA_general=tracked_AREA(type='shallow',type2='nonSBZ',updraft_type='general')
nonSBZ_SHALLOW_profile_array_AREA_cloudy=tracked_AREA(type='shallow',type2='nonSBZ',updraft_type='cloudy')

print('\n'+'DEEP')
SBZ_DEEP_profile_array_AREA_general=tracked_AREA(type='deep',type2='SBZ',updraft_type='general')
SBZ_DEEP_profile_array_AREA_cloudy=tracked_AREA(type='deep',type2='SBZ',updraft_type='cloudy')

nonSBZ_DEEP_profile_array_AREA_general=tracked_AREA(type='deep',type2='nonSBZ',updraft_type='general')
nonSBZ_DEEP_profile_array_AREA_cloudy=tracked_AREA(type='deep',type2='nonSBZ',updraft_type='cloudy')



In [ ]:
#SAVING

profile_dict = {}

categories = ['ALL', 'SHALLOW', 'DEEP']
types2 = ['SBZ', 'nonSBZ']
updraft_types = ['general', 'cloudy']

for category in categories:
    print(category)  # Print category name
    for type2 in types2:
        for updraft_type in updraft_types:
            var_name = f"{type2}_{category}_profile_array_AREA_{updraft_type}"
            profile_dict[var_name] = eval(var_name)  # Dynamically reference existing variables

dir2=dir+'Project_Algorithms/Tracked_Profiles/job_out2/'
output_file=dir2+f"SBZ_nonSBZ_tracked_Area_profiles_{res}_{t_res}_{Np_str}"
if job_array==True:
    output_file+=f"_{job_id}.h5"
elif job_array==False:
    output_file+=f".h5"
with open(output_file, "wb") as f:
    pickle.dump(profile_dict, f)

In [ ]:
#########################################
#RECOMBINE SEPERATE JOB_ARRAYS AFTER
recombine=False #KEEP FALSE WHEN JOBARRAY IS RUNNING
# recombine=True

In [ ]:
if recombine==True:
    def load_dict(job_id):
        dir2=dir+'Project_Algorithms/Tracked_Profiles/job_out2/'
        input_file=dir2+f"SBZ_nonSBZ_tracked_Area_profiles_{res}_{t_res}_{Np_str}_{job_id}.h5"
        with open(input_file, "rb") as f:
            profile_dict = pickle.load(f)
        return profile_dict
    
    job_id=1
    profile_dict1 = load_dict(job_id)
    
    num_jobs=300
    for job_id in np.arange(1,num_jobs+1):
        profile_dict2 = load_dict(job_id)
        if np.mod(job_id,10)==0: print(f"job_id = {job_id}")
        for var in profile_dict2:
            profile_dict1[var][:,0:1+1]+=profile_dict2[var][:,0:1+1]
    
    dir2=dir+'Project_Algorithms/Tracked_Profiles/OUTPUT_FILES/'
    output_file=dir2+f"SBZ_nonSBZ_tracked_Area_profiles_{res}_{t_res}_{Np_str}.h5"
    with open(output_file, "wb") as f:
        pickle.dump(profile_dict1, f)

In [ ]:
#######################
#COLD POOL

In [ ]:
################################
#CALCULATING

In [ ]:
print('\n'+'ALL')
ColdPool_ALL_profile_array_AREA_general=tracked_AREA(type='all',type2='ColdPool',updraft_type='general')
ColdPool_ALL_profile_array_AREA_cloudy=tracked_AREA(type='all',type2='ColdPool',updraft_type='cloudy')

print('\n'+'SHALLOW')
ColdPool_SHALLOW_profile_array_AREA_general=tracked_AREA(type='shallow',type2='ColdPool',updraft_type='general')
ColdPool_SHALLOW_profile_array_AREA_cloudy=tracked_AREA(type='shallow',type2='ColdPool',updraft_type='cloudy')


print('\n'+'DEEP')
ColdPool_DEEP_profile_array_AREA_general=tracked_AREA(type='deep',type2='ColdPool',updraft_type='general')
ColdPool_DEEP_profile_array_AREA_cloudy=tracked_AREA(type='deep',type2='ColdPool',updraft_type='cloudy')

In [ ]:
#SAVING

profile_dict = {}

categories = ['ALL', 'SHALLOW', 'DEEP']
types2 = ['ColdPool']
updraft_types = ['general', 'cloudy']

for category in categories:
    print(category)  # Print category name
    for type2 in types2:
        for updraft_type in updraft_types:
            var_name = f"{type2}_{category}_profile_array_AREA_{updraft_type}"
            profile_dict[var_name] = eval(var_name)  # Dynamically reference existing variables

dir2=dir+'Project_Algorithms/Tracked_Profiles/job_out2/'
output_file=dir2+f"ColdPool_tracked_Area_profiles_{res}_{t_res}_{Np_str}"
if job_array==True:
    output_file+=f"_{job_id}.h5"
if job_array==False:
    output_file+=f".h5"
with open(output_file, "wb") as f:
    pickle.dump(profile_dict, f)

In [ ]:
#########################################
#RECOMBINE SEPERATE JOB_ARRAYS AFTER

In [ ]:
recombine=False #KEEP FALSE WHEN JOBARRAY IS RUNNING
recombine=True

In [ ]:
if recombine==True:
    def load_dict(job_id):
        dir2=dir+'Project_Algorithms/Tracked_Profiles/job_out2/'
        input_file=dir2+f"ColdPool_tracked_Area_profiles_{res}_{t_res}_{Np_str}_{job_id}.h5"
        with open(input_file, "rb") as f:
            profile_dict = pickle.load(f)
        return profile_dict
    
    job_id=1
    profile_dict1 = load_dict(job_id)
    
    num_jobs=300
    for job_id in np.arange(1,num_jobs+1):
        profile_dict2 = load_dict(job_id)
        if np.mod(job_id,10)==0: print(f"job_id = {job_id}")
        for var in profile_dict2:
            profile_dict1[var][:,0:1+1]+=profile_dict2[var][:,0:1+1]
    
    dir2=dir+'Project_Algorithms/Tracked_Profiles/OUTPUT_FILES/'
    output_file=dir2+f"ColdPool_tracked_Area_profiles_{res}_{t_res}_{Np_str}.h5"
    with open(output_file, "wb") as f:
        pickle.dump(profile_dict1, f)